In [1]:
#notes, links, etc.

#FRED data: 
#dataset: https://fred.stlouisfed.org/series/CCUSMA02CZM618N#
#link structure API: https://api.stlouisfed.org/fred/series?series_id=GNPCA&api_key=abcdefghijklmnopqrstuvwxyz123456&file_type=json
key = '290c57887172f5dd570f2f3e19b006dd'
series_ID = 'CCUSMA02CZM618N'

#COVID-Data
covid_url = 'https://covid.ourworldindata.org/data/owid-covid-data.csv'

#GDP Data
gdp_url = 'https://tradingeconomics.com/czech-republic/gdp-constant-prices'

In [2]:
#import packages
import requests
import pandas as pd

In [3]:
#Requests class: initialized with url to get data from and request type (kind)

class Req:
    def __init__(self, url, kind, data = None):
        self.url = url
        self.kind = kind
        
#class method to request data from url depending on type of requests (API, Direct Download, ...)
#note API request yet specifically written for FRED request -> should be more general
        
    def request_data(self, index):
        if self.kind == 'DirectDl':
            self.data = pd.read_csv(self.url).set_index(index)
        elif self.kind == 'API':
            self.data = pd.DataFrame(requests.get(self.url).json()['observations']).set_index([index]).drop(['realtime_start', 'realtime_end'], axis=1)
        else:
            print('Request-type not eglible.')
        pass

In [4]:
#create instances of Req for FRED and OWID request
FRED_Req = Req('https://api.stlouisfed.org/fred/series/observations?series_id=CCUSMA02CZM618N&api_key=290c57887172f5dd570f2f3e19b006dd&file_type=json', 'API')
OWID_Req = Req('https://covid.ourworldindata.org/data/owid-covid-data.csv', 'DirectDl')

In [5]:
#request data from FRED + OWID, save as df + covid_df
OWID_Req.request_data('date')
FRED_Req.request_data('date')

czk_df = FRED_Req.data
covid_df = OWID_Req.data

In [11]:
print(czk_df)
czk_df.to_parquet('FRED_Req.parquet')

                       value
date                        
1991-01-01  27.6468180736364
1991-02-01     27.2440000535
1991-03-01  28.7399998619048
1991-04-01  29.9433334904762
1991-05-01  30.1157142085714
...                      ...
2020-12-01  21.6299130434783
2021-01-01  21.4747142857143
2021-02-01           21.3881
2021-03-01  22.0020434782609
2021-04-01  21.6945909090909

[364 rows x 1 columns]


In [12]:
print(covid_df)
covid_df.to_parquet('OWID_Req.parquet')

           iso_code continent     location  total_cases  new_cases  \
date                                                                 
2020-02-24      AFG      Asia  Afghanistan          1.0        1.0   
2020-02-25      AFG      Asia  Afghanistan          1.0        0.0   
2020-02-26      AFG      Asia  Afghanistan          1.0        0.0   
2020-02-27      AFG      Asia  Afghanistan          1.0        0.0   
2020-02-28      AFG      Asia  Afghanistan          1.0        0.0   
...             ...       ...          ...          ...        ...   
2021-05-19      ZWE    Africa     Zimbabwe      38612.0       17.0   
2021-05-20      ZWE    Africa     Zimbabwe      38635.0       23.0   
2021-05-21      ZWE    Africa     Zimbabwe      38664.0       29.0   
2021-05-22      ZWE    Africa     Zimbabwe      38679.0       15.0   
2021-05-23      ZWE    Africa     Zimbabwe      38682.0        3.0   

            new_cases_smoothed  total_deaths  new_deaths  new_deaths_smoothed  \
date    